# Imports

In [1]:
import teslapy
import pandas as pd
import time
from datetime import datetime


# Connect to Tesla

In [2]:
# Authenticate
tesla = teslapy.Tesla('ssopariwalla@gmail.com')
if not tesla.authorized:
    tesla.fetch_token()

# Select vehicle
vehicle = tesla.vehicle_list()[0]
vehicle.sync_wake_up()

Use browser to login. Page Not Found will be shown at success.
Open this URL to authenticate: https://auth.tesla.com/oauth2/v3/authorize?response_type=code&client_id=ownerapi&redirect_uri=https%3A%2F%2Fauth.tesla.com%2Fvoid%2Fcallback&scope=openid+email+offline_access&state=HhZCo1JTqSCCD3iA9MVA0oJUAdkTsl&code_challenge=6Pw07REStWSxB0hIAqmvrH315hiOxfCLr360uRl-Cvk&code_challenge_method=S256&login_hint=ssopariwalla%40gmail.com


Enter URL after authentication:  https://auth.tesla.com/void/callback?code=NA_e3a156673f6ff5227c155fa033a90b6b90c1b9d9a378413e93bdf39b53ec&state=HhZCo1JTqSCCD3iA9MVA0oJUAdkTsl&issuer=https%3A%2F%2Fauth.tesla.com%2Foauth2%2Fv3


In [3]:
log_path = 'tesla_odometer_log.csv'

while True:
    try:
        data = vehicle.get_vehicle_data()
        odometer = data['vehicle_state']['odometer']
        drive_state = data.get('drive_state', {})
        latitude = drive_state.get('latitude')
        longitude = drive_state.get('longitude')
        timestamp = datetime.utcnow().isoformat()

        # Append to CSV
        row = pd.DataFrame([{
            'timestamp': timestamp,
            'odometer_miles': odometer,
            'latitude': latitude,
            'longitude': longitude
        }])
        row.to_csv(log_path, mode='a', header=not pd.io.common.file_exists(log_path), index=False)

        print(f"{timestamp} | Odometer: {odometer} miles | Location: ({latitude}, {longitude}) (saved)", end='\r', flush=True)

        time.sleep(30)

    except Exception as e:
        print(f"\nError: {e}")
        time.sleep(60)  # Wait 1 minute before retry


2025-05-01T06:21:01.309423 | Odometer: 6987.203192 miles (saved)


KeyboardInterrupt

